In [1]:
# Atnaujintas variantas, po to, kaip padaryti pakeitimai failų varduose 2022 03 26
# Šitame variante keičiamas list.json ir df_transl failai
#
# iš originalių Zive įrašų sukuriami įrašai *.npy,
# Zive ekg įrašo identikatoriai userId, file_name pakeičiami į virtualią numeraciją:
# userId pakeičiamas į userNr - skaičiumi, pradedant nuo 1000, įrašo įdentifikatorius file_name pakeičiamas
# į įrašo eilės numerį registrationNr, lygų 0,1,... ir suformuojamas virtualus įrašo failo vardas SubjCode.
# Konversijai ir saugojimui panaudojamas df masyvas df_transl.
# Koreguojamas list.json failas, papildant atributu SubjCode. Anotacijų failai *.json - nekeičiami.


import shutil, sys
from icecream import ic
from pathlib import Path
import pandas as pd
import numpy as np

import json
from zive_util_vu import zive_read_file_1ch, create_dir, create_SubjCode, get_recNr


import warnings
warnings.filterwarnings("ignore")

my_os=sys.platform
print("OS in my system : ",my_os)

if my_os != 'linux':
    OS = 'Windows'
else:  
    OS = 'Ubuntu'

# Bendras duomenų aplankas, kuriame patalpintas subfolderis name_db

if OS == 'Windows':
    Duomenu_aplankas = 'D:\DI\Data\MIT&ZIVE\VU'   # variantas: Windows
else:
    Duomenu_aplankas = '/home/kesju/DI/Data/MIT&ZIVE/VU'   # arba variantas: UBUNTU, be Docker

# jei variantas Docker pasirenkame:
# Duomenu_aplankas = '/Data/MIT&ZIVE'

# Vietinės talpyklos aplankas
db_folder = 'DUOM_VU'

# Nuoroda į aplanką su EKG duomenų rinkiniu
db_path = Path(Duomenu_aplankas, db_folder)

rec_dir = Path(db_path,'records_selected')
rec_list = 'list_tst.json'

# Nuoroda į aplanką su transformuotu EKG duomenų rinkiniu
rec_dir_npy = Path(db_path, 'records_npy')
rec_list_npy = 'list_tst_npy.json'

# Paliekamų anotacijų sąrašas
annot_list = ['N','S','V','U']

# Diskretizavimo dažnis
fs_zive = 200

print("\nSkriptas zive įrašų transformacijai į npy formatą\n")

print("Išeities duomenys skaitomi iš:", rec_dir)
print("EKG įrašų atributų sąrašas:", rec_list)

print("Transformuoti duomenys rašomi į:", rec_dir_npy)
print("Atnaujintas EKG įrašų atributų sąrašas:", rec_list_npy)
print("\n")


OS in my system :  win32

Skriptas zive įrašų transformacijai

Išeities duomenys skaitomi iš: D:\DI\Data\MIT&ZIVE\VU\DUOM_VU\records_selected
EKG įrašų atributų sąrašas: list_tst.json
Transformuoti duomenys rašomi į: D:\DI\Data\MIT&ZIVE\VU\DUOM_VU\records_npy
Atnaujintas EKG įrašų atributų sąrašas: list_tst_npy.json




In [2]:

# Sukūriame aplanką EKG sekų įrašymui
create_dir(rec_dir_npy)

pd.set_option("display.max_rows", 1000, "display.max_columns", 20)
pd.set_option('display.width', 2000)

# Nuskaitome Zive įrašų talpykloje laikomų įrašų sąrašą
file_path = Path(rec_dir, rec_list)
with open(file_path,'r', encoding='UTF-8', errors = 'ignore') as f:
    data = json.loads(f.read())

df_list = pd.json_normalize(data, record_path =['data'])

# print(df_list)
# Sugrupuojame įrašus į to paties paciento grupes
grouped = df_list.groupby(['userId','file_name'])
print(grouped)
print("\n")

SubjCodes = []

# Perrašome įrašus nauju formatu į kitą folderį
for key in grouped.groups:
    userId = key[0]
    file_name = str(key[1])
    
    userNr, recordingNr = get_recNr(rec_dir_npy, userId, file_name)
    SubjCode = create_SubjCode(userNr, recordingNr)  
    # print("SubjCode: ", SubjCode, userNr, file_name)

    SubjCodes.append(SubjCode)

    file_path = Path(rec_dir, file_name)
    signal = zive_read_file_1ch(file_path)  

    file_path = Path(rec_dir_npy, str(SubjCode) + '.npy')
    with open(file_path, 'wb') as f:
        np.save(f, signal)

    src = Path(rec_dir, file_name + '.json')
    dst = Path(rec_dir_npy, str(SubjCode) + '.json')
    shutil.copy2(src, dst)

print(SubjCodes)

# Papildome įrašų sarašą df_list stulpeliu SubjCode
df_list.insert(0, 'SubjCode', SubjCodes, True)

# Įrašome failų sąrašą į diską
file_path = Path(rec_dir_npy, rec_list_npy)       
df_list.to_json(file_path, orient = 'table', index=False)
print(f'\nEKG įrašų atributų sąrašas įrašytas:  {file_path}')


info = {
    'db_folder':db_folder,
    'fs': fs_zive,
    'SubjCodes':SubjCodes,
    'annot_list':annot_list
    }

file_name = Path(rec_dir_npy,'info_create_z.json')
with open(file_name, 'w') as f:
    json.dump(info, f)
    
print("\nSchemos parametrai įrašyti į failą: ", file_name, "\n" )



Directory 'D:\DI\Data\MIT&ZIVE\VU\DUOM_VU\records_npy' already exists


[10000, 10010, 10020, 10030, 10031, 10032, 10040, 10050, 10051, 10052, 10053, 10060, 10070, 10080, 10090, 10091, 10100, 10110, 10111, 10120, 10130]

EKG įrašų atributų sąrašas įrašytas:  D:\DI\Data\MIT&ZIVE\VU\DUOM_VU\records_npy\list_tst_npy.json

Schemos parametrai įrašyti į failą:  D:\DI\Data\MIT&ZIVE\VU\DUOM_VU\records_npy\info_create_z.json 



In [3]:
# https://www.geeksforgeeks.org/python-shutil-copy2-method/#:~:text=shutil.copy
# 2%20%28%29%20method%20in%20Python%20is%20used%20to,destination%20can%20be%20a%20file%20or%20a%20directory.

# Susirandame anotacijų pasiskirstymą per pacientus ir pacientų skaičių
df_sum = df_list.groupby(['userId'],sort = False).sum()
# print(df_sum)
# https://sparkbyexamples.com/pandas/pandas-groupby-sum-examples/
count = df_list['userId'].value_counts()
print(f'\nViso pacientų: {len(count)}  EKG įrašų: {len(df_list)}')
print(f'\nĮrašų pasiskirstymas per pacientus')
count = count.rename("count")
frames = [df_sum, count]
result = pd.concat(frames, axis = 1)
result.index.rename ('userID', inplace= True)
result = result.reset_index()

result.drop(labels=['incl'], axis=1, inplace=True)
result.drop(labels=['flag'], axis=1, inplace=True)
result.drop(labels=['SubjCode'], axis=1, inplace=True)
# https://www.shanelynn.ie/pandas-drop-delete-dataframe-rows-columns/
print(result)

suma = result.iloc[:,1:].sum()
print('\n',' '*5, 'sum: ',str(suma.tolist())[1:-1])
total = suma.sum()   # pataisyti, kad sumuotų tik pūpsnius
print(' '*5, 'total: ', total)



Viso pacientų: 14  EKG įrašų: 21

Įrašų pasiskirstymas per pacientus
                      userID     N    S    V  U  count
0   61632b451e325518bb017bbd   720    1    0  0      1
1   6190d4e63cd1d227c1303cea  1154    3    1  0      2
2   6144c588bd0cc52ba0275362  1558    0  122  0      3
3   61a8863e5cd547e980a4e970  1277  189   42  0      2
4   61632ace1e32557a62017bba  3920  321    3  0      4
5   6144c532bd0cc54c83275360   787    0    0  0      1
6   6143507abd0cc5051b275171   756    1    4  0      1
7   61b32023cf0f347968cb1c91   698    1    0  0      1
8   6144c682bd0cc5acb7275368   716    0   96  0      1
9   61b31ff1cf0f34275ccb1c90   637    2   74  0      1
10  619b4bbbb481320f924e3da9   645    1    0  0      1
11  616b00a811601579e9c2816c   821    0    0  0      1
12  6144c4fbbd0cc552e427535f   540    0  205  0      1
13  617509686d4fb48519637743   796   12    1  0      1

       sum:  15025, 531, 548, 0, 21
      total:  16125
